In [1]:
from igraph import *

In [2]:
edges = [(0, 1),
         (1, 2),
         (2, 3),
         (1, 4),
         (4, 5),
         (4, 6),
         (2, 7),
         (0, 8),
         (8, 9),
         (9, 10),
         (9, 11)]

tree = Graph(edges=edges)
tree.vs['label'] = [v.index for v in tree.vs]
#plot(tree,layout=tree.layout('rt',root=[0]),bbox=(300,300))

In [3]:
test_d = {'d1': [0, 0, 0],
          'd2': [0, 1, 0],
          'd3': [0, 1, 1],
          'd4': [0, 0, 1],
          'd5': [1, 1, 0],
          'd6': [1, 1, 1]}
test_d

{'d1': [0, 0, 0],
 'd2': [0, 1, 0],
 'd3': [0, 1, 1],
 'd4': [0, 0, 1],
 'd5': [1, 1, 0],
 'd6': [1, 1, 1]}

In [4]:
def find_maxes(test_d):
    compare_list = []
    D_list = {}
    D_count = 1
    d_count = 0

    for i in range(1, len(test_d)):
        temp = []
        sim_d = {}
        # compare_list.append(test_d['d'+str(i)])
        
        up = test_d['d'+str(i)]
        down = test_d['d'+str(i+1)]

        compare_list.append(up)
        for k in compare_list:
            up = k
            sim_score = 0
            for j in range(3):
                if down[j] == up[j]:
                    sim_score += 1
                else:
                    break
            sim_d[str(k)+str(down)] = sim_score
            d_count += 1

        D_list[str(D_count)] = sim_d
        D_count += 1

    return D_list


In [6]:
# function to return key for any value 
def get_key(New_DL):
    count = 0
    most_alike = []

    for j in New_DL:
        for key, value in j.items():
            if maxes_ind[count] == value:
                most_alike.append(key)
                count += 1
        
    return most_alike    

In [7]:
def ret_list(maxes_s):
    temp = []
    ML = []

    for i in range(5):
        for l in maxes_s[i]:
            if len(temp) < 3:
                if l=='0' or l=='1':
                    temp.append(int(l))
        ML.append(temp)
        temp = []

    return ML

In [42]:
def get_key(New_DL):
    count = 0
    most_alike = []

    for j in New_DL:
        for key, value in j.items():
            if maxes_ind[count] == value:
                most_alike.append(key)
                count += 1
        
    return most_alike    

In [43]:
get_key(New_DL)

['[0, 0, 0][0, 1, 0]',
 '[0, 1, 0][0, 1, 1]',
 '[0, 0, 0][0, 0, 1]',
 '[0, 0, 0][1, 1, 0]',
 '[1, 1, 0][1, 1, 1]']

In [44]:
D_list = find_maxes(test_d)
New_DL = []
for i in range(1, 6):
    New_DL.append(D_list[str(i)])

maxes_ind = []
for i in D_list.values():
    maxes_ind.append(max(i.values()))

maxes_s = get_key(New_DL)

MAX = ret_list(maxes_s)
MAX

[[0, 0, 0], [0, 1, 0], [0, 0, 0], [0, 0, 0], [1, 1, 0]]

In [ ]:
def all_same(items):
    return all(x == items[0] for x in items)

check_vals = []
for v in D_list['4'].values():
    check_vals.append(v)

all_same(check_vals)

In [13]:
ind_d = {'i1': [1, 2, 3]}
same_path = True
count = 3

for i in range(1, len(test_d)):
    temp = []

    up = MAX[i]
    down = test_d['d'+str(i+1)]

    for j in range(3):
        if down[j] == up[j] and same_path==True:
            temp.append(ind_d['i'+str(i)][j])
        else: 
            count += 1
            temp.append(count)
            same_path=False

    ind_d['i'+str(i+1)] = temp
    same_path=True

ind_d

IndexError: list index out of range

In [ ]:
import pprint
pprint.pprint(D_list)

In [ ]:
maxes_ind = []
for i in D_list.values():
    maxes_ind.append(max(i.values()))

maxes_s = []
for j in maxes_ind:
    maxes_s.append(get_key(j))

maxes_s
MAX = ret_list(maxes_s)
MAX

In [ ]:
def ret_list(maxes_s):
    temp = []
    ML = []

    for i in range(5):
        for l in maxes_s[i]:
            if len(temp) < 3:
                if l=='0' or l=='1':
                    temp.append(int(l))
        ML.append(temp)
        temp = []

    return ML